In [ ]:
import os
import pandas as pd
import pickle
from importlib import reload
import lime.lime_rdf
from rdflimeConfig import movieLocation
import logging 

logging.basicConfig(level=logging.WARN)

movieFull = pd.read_csv(os.path.join(movieLocation, "movies_fixed.tsv"), sep="\t")
movies = [movie.DBpedia_URI for _, movie in movieFull.iterrows()]

In [ ]:
reload(lime.lime_rdf)
from lime.lime_rdf import LimeRdfExplainer

with open(os.path.join(movieLocation, "transformers", "rdf2vec_transformer_cbow_200"), "rb") as file:
    rdf2vec_transformer = pickle.load(file)

with open(os.path.join(movieLocation, "classifiers", "svc_100_cbow_200"), "rb") as file:
    clf = pickle.load(file)

explainer = LimeRdfExplainer(
    transformer=rdf2vec_transformer, 
    entities=movies,
    class_names=clf.classes_,
    kernel=None,
    kernel_width=25,
    verbose=False,
    feature_selection="auto",
    random_state=42
)


explained_entity_id = 274  # 0-400 -> test data
explained_entity_uri = movies[explained_entity_id]
prediction = clf.predict_proba([rdf2vec_transformer._embeddings[explained_entity_id]])


print("Explaining", explained_entity_uri)
print("Original prediction:", prediction, " / ".join(clf.classes_))
print("True class:", movieFull.iloc[explained_entity_id].label)

data, probabilities, distances, explanation = explainer.explain_instance(
    entity=explained_entity_uri, 
    classifier_fn=clf.predict_proba,
    num_features=50,
    num_samples=5000,
    allow_triple_addition=False,
    allow_triple_subtraction=True,
    max_changed_triples=20,
    change_count_fixed=True,
    use_w2v_freeze=True,
    center_correction=False,
    single_run=False,
    train_with_all=False,
    distance_metric="cosine",
    model_regressor=None,
    short_uris=False
)

In [ ]:
figsize = (4, .4*len(explanation.as_list()))

explanation.domain_mapper.short_uris=True
fig = explanation.as_pyplot_figure(figsize=figsize)
explanation.domain_mapper.short_uris=False


In [ ]:
pd.DataFrame(probabilities).plot()

# thesis: show plot with/without single_run

In [ ]:
explanation.domain_mapper.short_uris=True
with open("test.html", "w") as f:
    f.write(explanation.as_html())
    explanation.domain_mapper.short_uris=False

In [ ]:
iw = explainer.indexed_walks

top_triples = explanation.as_list()[:15]

relevant_walks = []
for triple in top_triples:
    for w in iw.walks(explained_entity_uri, tuple(triple[0])):
        
        entity_pos = w.index(explained_entity_uri)

        for i in range(2, len(w)):
            if w[i-2] == triple[0][0] and w[i-1] == triple[0][1] and w[i] == triple[0][2]:
                triple_start_pos = i-2
                triple_end_pos = i
        
        from_pos = min(entity_pos, triple_start_pos)
        to_pos = max(entity_pos, triple_end_pos)

        relevant_walks.append(w[from_pos:to_pos+1])



relevant_triples = iw.walks_as_triples(relevant_walks)

In [ ]:
edges = []
labels = {}

min_edge_width = 0.25
max_edge_width = 8
max_score = max([abs(exp[1]) for exp in top_triples])

uri_trimmer = lambda uri: uri.split("/")[-1].split("#")[-1].replace(":", "_")
triple_trimmer = lambda triple: (uri_trimmer(triple[0]), uri_trimmer(triple[1]), uri_trimmer(triple[2]))

for rt in relevant_triples:
    score = next((exp[1] for exp in top_triples if rt==exp[0]), None)
    
    if score:
        # score *= -1
        color = "green" if score > 0 else "red"
        width = max(abs(score)/max_score*max_edge_width, min_edge_width)
    else:
        color = "blue"
        width = min_edge_width
        #print(rt, score)

    rt = triple_trimmer(rt)
    edge = (rt[0], rt[2], {"width": width, "color": color})
    
    label = rt[1]
    if score: label += f" ({round(width/max_edge_width, 2)})"

    edges.append(edge)
    
    labels[(edge[0],edge[1])] = label
    


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from math import sqrt


G = nx.DiGraph()
G.add_edges_from(edges)
#pos = nx.spring_layout(G, k=1.25/sqrt(len(G.nodes())), seed=42, iterations=35)
pos = nx.nx_pydot.graphviz_layout(G, prog="neato")
plt.figure(figsize=(18,18))
nx.draw(
    G, pos,
    node_size=500, node_color='lightblue', alpha=0.9,
    labels={node: node for node in G.nodes()},
    arrowstyle='->',
    width=[G[u][v]['width'] for u,v in G.edges()],
    edge_color=[G[u][v]['color'] for u,v in G.edges()],
)


_ = nx.draw_networkx_edge_labels(
    G, pos,
    edge_labels=labels,
    font_color='black',
)

# plt.savefig("green_lantern.pdf")

x = nx.nx_pydot.to_pydot(G)

# Bug: node labels with :

In [ ]:
def check_explanations():
    exp_list = explanation.as_list()[:25]

    for exp in exp_list:
        triple = exp[0]
        if "_" in triple: continue
        modified_walks = explainer.get_perturbed_walks(explained_entity_uri, added_triples=[], removed_triples=[triple])
        embedding = explainer.get_perturbed_embedding(explained_entity_uri, modified_walks)
        new_prediction = clf.predict_proba([embedding])
        print( (new_prediction[0][1] - prediction[0][1]) / exp[1], triple )

In [ ]:
#
# Explained Change / Real Change
#
# +0.5 / +0.5 
# +
#


In [ ]:
check_explanations()

### Experiments

In [ ]:
from pyvis.network import Network
nt = Network('500px', '500px')
nt.from_nx(G)
nt.show("nx.html")

In [ ]:
wv_old = explainer.old_transformer.embedder._model.wv
wv_new = explainer.transformer.embedder._model.wv
def embedding(URI, new):
    wv = wv_new if new else wv_old
    return wv.get_vector(URI)

In [ ]:
wv_old.most_similar(explainer.new_embeddings[150])

In [ ]:
# Check if the movie embedding itself drifted
all(embedding(explained_entity_uri, new=True) == embedding(explained_entity_uri, new=False))

In [ ]:
for walk in explainer.indexedWalks.walks(e):
    for step in walk:
        if not step in changed and not step == e:
            print(step)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
# Reduce the dimensions of entity embeddings to represent them in a 2D plane.
X_tsne = TSNE(random_state=42).fit_transform([wv_old[explained_entity_uri]] + explainer.new_embeddings)

labels = explainer.old_transformer._entities + list(range(len(explainer.new_embeddings)))
labels = [explained_entity_uri] + list(range(len(explainer.new_embeddings)))

#colors = list(map(lambda e: "#00ff00" if movieTest[movieTest.DBpedia_URI==e].iloc[0].label == "good" else "#ff0000", transformer._entities[1600:]))
#sizes = list(map(lambda e: abs(50-movieTest[movieTest.DBpedia_URI==e].iloc[0].rating)**2, transformer._entities[1600:]))
sizes = list(map(lambda l: 100**2 if l == explained_entity_uri or l == 0 else 25**2, labels))

# Ploy the embeddings of entities in a 2D plane, annotating them.
f = plt.figure(figsize=(200, 80))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], s=sizes)#, c=colors, s=sizes)

for x, y, t in zip(X_tsne[:, 0], X_tsne[:, 1], labels):
    if type(t) == str and "http" in t: t = t.split("/")[-1]
    plt.annotate(t, (x, y))

# Display the graph with a title, removing the axes for better readability.
plt.title("pyRDF2Vec", fontsize=4)
plt.axis("off")
plt.show()

#f.savefig("figure.pdf", bbox_inches='tight')


In [ ]:
unique_triples = len(explainer.indexed_walks.walks_as_triples(explainer.indexed_walks.walks(explained_entity_uri)))
unique_triples

In [ ]:
keanu = embedding("http://dbpedia.org/resource/Keanu_Reeves", new=True)
matrix = embedding("http://dbpedia.org/resource/The_Matrix", new=True)
wv_new.most_similar(matrix-keanu, topn=len(wv_new))[0]
wv_new.most_similar(positive=["http://dbpedia.org/resource/The_Matrix"], negative=["http://dbpedia.org/resource/Keanu_Reeves"])

In [ ]:
movie_search = lambda query: [(i,x) for i,x in enumerate(movies) if query.lower() in x.lower()]
movie_search("star")

In [ ]:
movieFull[200:220]

In [ ]:
from concurrent.futures import ProcessPoolExecutor

def xm(i): 
    print(i)

with ProcessPoolExecutor() as executor:
    executor.map(xm, [1,2,3])

In [ ]:
for i in range(155):

    reload(lime.lime_rdf)
    from lime.lime_rdf import LimeRdfExplainer

    with open(os.path.join(movieLocation, "transformers", "rdf2vec_transformer_cbow_200"), "rb") as file:
        rdf2vec_transformer = pickle.load(file)

    with open(os.path.join(movieLocation, "classifiers", "svc_100_cbow_200"), "rb") as file:
        clf = pickle.load(file)

    explainer = LimeRdfExplainer(
        transformer=rdf2vec_transformer, 
        entities=movies,
        class_names=clf.classes_,
        kernel=None,
        kernel_width=25,
        verbose=False,
        feature_selection="auto",
        random_state=42
    )


    explained_entity_id = i  # 0-400 -> test data
    explained_entity_uri = movies[explained_entity_id]
    prediction = clf.predict_proba([rdf2vec_transformer._embeddings[explained_entity_id]])


    print("Explaining", explained_entity_uri)
    print("Original prediction:", prediction, " / ".join(clf.classes_))
    print("True class:", movieFull.iloc[explained_entity_id].label)

    data, probabilities, distances, explanation = explainer.explain_instance(
        entity=explained_entity_uri, 
        classifier_fn=clf.predict_proba,
        num_features=50,
        num_samples=5000,
        allow_triple_addition=False,
        allow_triple_subtraction=True,
        max_changed_triples=20,
        change_count_fixed=True,
        use_w2v_freeze=True,
        center_correction=False,
        single_run=False,
        train_with_all=False,
        distance_metric="cosine",
        model_regressor=None,
        short_uris=False
    )

    with open(f"explanation-{i}", "wb") as f:
        pickle.dump([explanation, probabilities], f)